In [1]:
!pip uninstall -y langchain langchain-core langchain-community langchain-google-genai langchain-groq
!pip install -U langchain langchain-core langchain-community langchain-google-genai langchain-groq
!pip install PyMuPDF faiss-cpu

Found existing installation: langchain 1.1.3
Uninstalling langchain-1.1.3:
  Successfully uninstalled langchain-1.1.3
Found existing installation: langchain-core 1.1.3
Uninstalling langchain-core-1.1.3:
  Successfully uninstalled langchain-core-1.1.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.1/476.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
   

In [3]:
# # Tidak perlu dijalankan
# from google.colab import userdata

# GOOGLE_API_KEY = userdata.get('gemini_key')
# GROQ_API_KEY = userdata.get('groq')

In [ ]:
GOOGLE_API_KEY = "google-api-key-anda"
GROQ_API_KEY = "groq-api-key-anda"

In [7]:
import os

# Masukkan Google API key anda
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
# Masukkan Groq key anda
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

## Simple Document Reading

In [20]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct")

In [9]:
TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [11]:
example_prompt = TEMPLATE.format(question="What is AI?", context="AI is something")
print(example_prompt)


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: What is AI?
Context: AI is something
Answer:



In [14]:
import fitz

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text() for page in doc])
    return text

In [16]:
pdf_text = extract_text_from_pdf("sample-resume.pdf")
print(pdf_text)

Functional Resume Sample 
 
John W. Smith  
2002 Front Range Way Fort Collins, CO 80525  
jwsmith@colostate.edu 
 
Career Summary 
 
Four years experience in early childhood development with a diverse background in the care of 
special needs children and adults.  
  
Adult Care Experience 
 
• Determined work placement for 150 special needs adult clients.  
• Maintained client databases and records.  
• Coordinated client contact with local health care professionals on a monthly basis.     
• Managed 25 volunteer workers.     
 
Childcare Experience 
 
• Coordinated service assignments for 20 part-time counselors and 100 client families. 
• Oversaw daily activity and outing planning for 100 clients.  
• Assisted families of special needs clients with researching financial assistance and 
healthcare. 
• Assisted teachers with managing daily classroom activities.    
• Oversaw daily and special student activities.     
 
Employment History  
 
1999-2002 
Counseling Supervisor, The Wesley

In [24]:
QUESTION = """
In scale 0 - 10, how suitable this person for the role of kindergarten teacher? Return in number
"""

prompt = TEMPLATE.format(question=QUESTION, context=pdf_text)
print(prompt)


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: 
In scale 0 - 10, how suitable this person for the role of kindergarten teacher? Return in number

Context: Functional Resume Sample 
 
John W. Smith  
2002 Front Range Way Fort Collins, CO 80525  
jwsmith@colostate.edu 
 
Career Summary 
 
Four years experience in early childhood development with a diverse background in the care of 
special needs children and adults.  
  
Adult Care Experience 
 
• Determined work placement for 150 special needs adult clients.  
• Maintained client databases and records.  
• Coordinated client contact with local health care professionals on a monthly basis.     
• Managed 25 volunteer workers.     
 
Childcare Experience 
 
• Coordinated service assignments for 20 part-time counselors and 100 client familie

In [25]:
from IPython.display import Markdown

response = llm.invoke(prompt)
Markdown(response.content)

I would rate this person a 6 out of 10 for the role of a kindergarten teacher. The person has experience working with special needs children and adults, and has a background in early childhood development with a BS in Early Childhood Development and a BA in Elementary Education. However, their experience seems more focused on adult care and supervisory roles rather than direct teaching experience.

## RAG

### Step 1: Split Document into Chunks

In [26]:
pdf_text = extract_text_from_pdf("baliguide-sample-pdf.pdf")

In [28]:
from langchain_text_splitters import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
)

chunks = splitter.split_text(pdf_text)

In [29]:
len(chunks)

70

In [34]:
from langchain_core.documents import Document

chunk_documents = [Document(page_content=chunk) for chunk in chunks]

Step 2: Get Chunks Embeddings

In [35]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [37]:
len(embedding_model.embed_query("Makan malam"))

3072

In [38]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = IndexFlatL2(
    len(embedding_model.embed_query("hello world"))
)
vector_db = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [39]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(chunk_documents))]

doc_ids = vector_db.add_documents(documents=chunk_documents, ids=uuids)

### Step 3: RAG

In [40]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile")

In [41]:
QUESTION = """
How can I commute to Uluwatu in Bali?
"""

In [47]:
relevant_docs = vector_db.similarity_search(QUESTION, k=3)

In [51]:
relevant_docs[2].page_content

"22 \nScooter rental costs 5-8US$ per day. Always put safety and reliability first. \nNever compromise. Enough people have serious or fatal accidents. Never hire \nrun down scooters to save a buck.\n23 \nKeep the mobile no. of the bike rental shop with you. You might need it.\n24 \nCar with driver for 8-10 hours costs 40-70US$ per day. Make sure they \nhave a driver's license and the permission to be a driver (izin, insurance issues) \nto avoid trouble with the police.\n25 \nTaxis are safe and cheap, but ask them to switch on the meter, always!\n26 \nUbud no meter taxis available, transport is organized by licensed drivers \nfrom the village. It's important to bargain!\n27 \nRenting a scooter or car? Bring international driving license\n28 \nYes, there is UBER and GRAB in Bali. But in many areas drivers are afraid to go \nto, because the local communities are against it and create major trouble.\n29 \nMotorbike Taxis can be arranged through the app Go-Jek. Works well, although"

In [52]:
docs_content = "\n\n".join(doc.page_content for doc in relevant_docs)
messages = TEMPLATE.format(
    context=docs_content, question=QUESTION
)
print(messages)


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: 
How can I commute to Uluwatu in Bali?

Context: you from A to B, apart from taxis. 
In major areas like Kuta, Seminyak or Sanur, you can walk around by foot quite easily. 
The distance between one touristic area to another can be quite far at times, and the 
heat, traffic, and sidewalk conditions can be stressful. If your hotel or villa is not very 
 Bali Essential Travel Guide/Strobel      40
near to the beach or restaurants, then you need some form of transportation to get 
there.
Almost everywhere on Bali, you will be continuously asked if you need transport. Be 
prepared to hear, 'transport? transport?' very often. It sometimes seems, as if every 
Balinese is a driver, who owns a car or a motorbike. Even though, it can get a bit tiring 

In [54]:
response = llm.invoke(messages)
print(response.content)

To commute to Uluwatu in Bali, you can use various transportation options such as taxis, renting a scooter, or hiring a car with a driver. It's recommended to bargain if the service is not priced through an app or with a taxi meter. Additionally, ride-hailing apps like Uber and Grab are available, but their availability may be limited in some areas.
